In [1]:
'''
generate dataset from qplib
from Xi Gao
'''

import osqp
import time
import tqdm
import random
import pickle
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix
import sys
import os

sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from dcopf_utils import DcopfProblem

## Import Data

In [22]:
# qp_path = ''
nbus = 8845
m = gp.read(f'/home/jxxiong/A-xjx/DC3/datasets/qplib/QPLIB_{nbus}.lp')
n_samples = 20
perturb_rate = 0.3

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/qplib/QPLIB_8845.lp
Reading time = 0.01 seconds
obj: 777 rows, 1546 columns, 10247 nonzeros


In [23]:
def get_constraints(m):
    ini_ineq_matrix = []
    ini_ineq_rhs = []
    ini_eq_matrix = []
    ini_eq_rhs = []
    a_matrix = m.getA().toarray()

    i = 0
    for constr in m.getConstrs():
        if constr.sense == '<':
            ini_ineq_matrix.append(a_matrix[i,:])
            ini_ineq_rhs.append(constr.RHS)
            i += 1
        elif constr.sense == '>':
            ini_ineq_matrix.append(-a_matrix[i,:])
            ini_ineq_rhs.append(-constr.RHS)
            i += 1
        elif constr.sense == '=':
            ini_eq_matrix.append(a_matrix[i,:])
            ini_eq_rhs.append(constr.RHS)
            i += 1

    ini_ineq_matrix = np.array(ini_ineq_matrix)
    ini_ineq_rhs = np.array(ini_ineq_rhs)
    ini_eq_matrix = np.array(ini_eq_matrix)
    ini_eq_rhs = np.array(ini_eq_rhs)

    return ini_ineq_matrix, ini_ineq_rhs, ini_eq_matrix, ini_eq_rhs

def get_objective(m):
    ini_q_matrix = m.getQ().toarray()
    ini_p_vec = []
    variables = m.getVars()
    for var in variables:
        ini_p_vec.append(var.obj)
    ini_p_vec = np.array(ini_p_vec)

    return ini_q_matrix, ini_p_vec

def get_bound(m):
    variables = m.getVars()
    ini_lb = []
    ini_ub = []
    for var in variables:
        ini_lb.append(var.lb)
        ini_ub.append(var.ub)
    ini_lb = np.array(ini_lb)
    ini_ub = np.array(ini_ub)

    return ini_lb, ini_ub

In [24]:
def perturb_eq_rhs(ini_eq_rhs, perturb_rate=perturb_rate):
    eq_rhs = ini_eq_rhs.copy()
    nonzero_indices = np.nonzero(ini_eq_rhs)
    nonzero_values = ini_eq_rhs[nonzero_indices]
    perturbed_values = nonzero_values * (1 + np.random.uniform(-perturb_rate, perturb_rate, size=nonzero_values.shape))
    eq_rhs[nonzero_indices] = perturbed_values
    return eq_rhs

In [25]:
def build_gurobi_model(Q, p, A, x, G, h, lb, ub):
    model = Model("qp")
    model.setParam('OutputFlag', 1)
    model.setParam('FeasibilityTol', 1e-4)

    n_vars = Q.shape[0]
    vars = []
    for i in range(n_vars):
        vars.append(model.addVar(lb=lb[i], ub=ub[i], vtype=GRB.CONTINUOUS, name=f"x_{i}"))

    obj = QuadExpr()
    for i in range(n_vars):
        for j in range(n_vars):
            if Q[i, j] != 0:
                obj.add(vars[i] * vars[j] * Q[i, j]* 0.5)

    for i in range(n_vars):
        if p[i] != 0:
            obj.add(vars[i] * p[i])

    model.setObjective(obj, GRB.MINIMIZE)

    for i in range(A.shape[0]):
        expr = LinExpr()
        for j in range(n_vars):
            if A[i, j] != 0:
                expr.add(vars[j] * A[i, j])
        model.addConstr(expr == x[i])
    
    for i in range(G.shape[0]):
        expr = LinExpr()
        for j in range(n_vars):
            if G[i, j] != 0:
                expr.add(vars[j] * G[i, j])
        model.addConstr(expr <= h[i])
    
    return model

In [26]:
def solve_osqp(Q, p, A, x, G, h, lb, ub):
    solver = osqp.OSQP()
    my_A = np.vstack([A, G, np.diag(np.ones(Q.shape[0]))])
    my_l = np.hstack([x, -np.ones(h.shape[0]) * np.inf, lb])
    my_u = np.hstack([x, h, ub])
    solver.setup(P=csc_matrix(Q), q=p, A=csc_matrix(my_A), l=my_l, u=my_u, verbose=True, eps_prim_inf=1e-10, eps_dual_inf=1e-10, eps_abs=1e-10, eps_rel=1e-10)
    results_osqp = solver.solve()
    sol_osqp = np.array(results_osqp.x)

    return sol_osqp

In [27]:
G, h, A, b = get_constraints(m)
Q, p = get_objective(m)
Lb, Ub = get_bound(m)

num_var = p.shape[0]
num_eq = A.shape[0]
num_ineq = G.shape[0]

print(f"number of variables: {num_var}, number of equality constraints: {num_eq}, number of inequality constraints: {num_ineq}")

number of variables: 1546, number of equality constraints: 490, number of inequality constraints: 287


In [8]:
X = []
Y = []
solve_time = []
for i in range(n_samples):
    print(i)
    x = perturb_eq_rhs(b)
    model = build_gurobi_model(Q, p, A, x, G, h, Lb, Ub)
    start_time = time.time()
    model.optimize()
    end_time = time.time()
    if model.status == GRB.OPTIMAL:
        sol = [var.x for var in model.getVars()]
        Y.append(sol)
        X.append(x)
        solve_time.append(end_time - start_time)

    if len(X) % 10 == 0:
        data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':Y, 'solve_time':solve_time}
        with open(f"dcopf{nbus}_data", 'wb') as f:
            pickle.dump(data, f)
    

0
Set parameter FeasibilityTol to value 0.0001
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 838 rows, 5223 columns and 15558 nonzeros
Model fingerprint: 0x4c3cc212
Model has 56975 quadratic objective terms
Coefficient statistics:
  Matrix range     [6e-03, 2e+00]
  Objective range  [7e+00, 6e+03]
  QObjective range [5e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-03, 3e+01]
Presolve removed 201 rows and 1048 columns
Presolve time: 0.01s
Presolved: 637 rows, 4175 columns, 10505 nonzeros
Presolved model has 44855 quadratic objective terms
Ordering time: 0.01s

Barrier statistics:
 Free vars  : 1670
 AA' NZ     : 1.016e+05
 Factor NZ  : 1.524e+05 (roughly 5 MB of memory)
 Factor Ops : 1.262e+07 (less than 1 second per iteration)
 Threads    : 24

  

KeyboardInterrupt: 

## Output data

In order to work with the varaible bounds, rename by dcopf

In [2]:
nbus = 8845
data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':[]}
with open(f"dcopf{nbus}_data", 'wb') as f:
    pickle.dump(data, f)

NameError: name 'Q' is not defined

In [ ]:
problem = DcopfProblem(Q, p, A, G, h, X)
problem.calc_Y()
print(len(problem.Y))

with open(f"dcopf{nbus}_data", 'wb') as f:
    pickle.dump(problem, f)